<a href="https://colab.research.google.com/github/Louisa328/Assignment-1/blob/main/Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Load your dataset here (ensure lalonde.csv is uploaded to Colab or linked)
df = pd.read_csv('lalonde.csv')

In [14]:
# Naive Comparison
naive_diff = df[df['treat']==1]['re78'].mean() - df[df['treat']==0]['re78'].mean()
print(f"Naive Difference in Means: ${naive_diff:.2f}")

Naive Difference in Means: $-635.03


In [20]:
# Define covariates
X = df[['age','educ','black','hispan','married','nodegree','re75','re78']]
y = df['treat']

# Fit Propensity Model
logit_v1 = LogisticRegression(solver='liblinear')
logit_v1.fit(X, y)

# Generate Scores
df['pscore'] = logit_v1.predict_proba(X)[:, 1]

# View the first few propensity scores
df[['treat','pscore']].head()

,treat,pscore
0,1,0.417834
1,1,0.133574
2,1,0.762270
3,1,0.664419
4,1,0.669873


In [21]:
from sklearn.neighbors import NearestNeighbors

# Separate groups
treated = df[df.treat==1]
control = df[df.treat==0]

# Fit NN on Control scores
nbrs = NearestNeighbors(n_neighbors=1).fit(control[['pscore']])

# Find matches for Treated scores
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()]

# Construct Matched DataFrame
matched_df = pd.concat([treated, matched_control])

In [22]:
from scipy import stats

# T-test on raw data
diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"Raw Effect (Difference): ${diff:,.2f}")
print(f"P-value: {p_val:.4f}")



# Isolate the matched outcomes
matched_treated = matched_df[matched_df.treat==1]['re78']
matched_control = matched_df[matched_df.treat==0]['re78']

# Estimate the causal effect (T-test on matched data)
matched_diff = matched_treated.mean() - matched_control.mean()
t_stat, p_val = stats.ttest_ind(matched_treated, matched_control)


print(f"Recovered Effect (Matched Difference): ${matched_diff:,.2f}")
print(f"P-value: {p_val:.4f}")

Raw Effect (Difference): $-635.03
P-value: 0.3342
Recovered Effect (Matched Difference): $1,850.03
P-value: 0.0109
